# Function Calling

In this lab, we'll work through a very basic function calling example. The 'agent' buzzword can feel big and scary, but here we'll see that it's a very straightforward tool in which we provide an LLM with explanations of our tools, and it chooses one (or more, or none!) to 'call'.

If you need any help during this lab, you can refer to the [Function calling guide here](https://platform.openai.com/docs/guides/function-calling).

## Step 1: Setup

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Load environment variables from .env file

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MODEL = "gpt-4o-mini"

## Step 2: Describing our Tools

We need to provide the agent with descriptions of the tools we want it to use. This helps the agent understand what each function does and what parameters it expects. We're intentionally doing this step _before_ we actually write the functions, because we want to prove once and for all that the LLM _isn't calling our functions_. It's just telling _us_ what we should call.

Though many of them are self explanatory, here's a breakdown of what each key does in the following tools:

- `"type": "function"`: This tool is a function.
- `"name": "add"`: The name of the function is `add`. Very important.
- `"description": "Add two numbers"`: This function adds two numbers.
- `"parameters": {...}`: Here's a dictionary containing information about what parameters the function takes.
- `"type": "object"`: This function takes a dictionary/object as an argument.
- `"properties": {...},`: This dictionary describes the properties of the dictionary that the function takes as an argument.
- `"a": {"type": "number", "description": "First number"}` & `"b": {"type": "number", "description": "Second number"}`: The dictionary should have two keys, `a`, and `b`, both of type number. `a` represents something called `"First number"` and `b` represents something called `"Second number"`.
- `"required": ["a", "b"]`: The object passed as an argument as a function is required to have both `a` and `b` properties.


In [ ]:
tools = [
    {
        "type": "function",
        "name": "add",
        "description": "Add two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "First number"},
                "b": {"type": "number", "description": "Second number"},
            },
            "required": ["a", "b"],
        },
    },
    {
        "type": "function",
        "name": "subtract",
        "description": "Subtract two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "Minuend"},
                "b": {"type": "number", "description": "Subtrahend"},
            },
            "required": ["a", "b"],
        }
    },
    {
        "type": "function",
        "name": "multiply",
        "description": "Multiply two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "First number"},
                "b": {"type": "number", "description": "Second number"},
            },
            "required": ["a", "b"],
        }
    },
    {
        "type": "function",
        "name": "divide",
        "description": "Divide two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "Dividend"},
                "b": {"type": "number", "description": "Divisor"},
            },
            "required": ["a", "b"],
        }
    },
]

## Step 3: Function Calling

Remember, we said we were going to use the tools calling feature before we actually define the functions.

In [ ]:
messages = [{"role": "user", "content": "Add three plus five"}]

response = client.responses.create(
    model=MODEL,
    input=messages,
    tools=tools,
)

# Everything except `response` is just for pretty printing
print(json.dumps(response.model_dump(), indent=2))

If everything has gone well, you should see a rather large object. Read through it and try to understand what each part is telling us. Most of the bulk is just our list of tools that we passed in.

The most important bit looks like this:

```json
  "output": [
    {
      "arguments": "{\"a\":3,\"b\":5}",
      "call_id": "call_IDYbPqowlyfI5xuFYBNyLKM9",
      "name": "add",
      "type": "function_call",
      "id": "fc_68a959e891ac8194a68ddd1194a9fef90ecfe6806f7bf2f7",
      "status": "completed"
    }
  ],
```

This section is where the model is telling us what function it wants to call and with what inputs. Let’s break it down:

1. `"arguments": "{\"a\":3,\"b\":5}"`

The model decided that the correct inputs are `a = 3` and `b = 5`.

Notice the arguments are a JSON string, so you’ll typically need to `json.loads` them in your code before using.

2. `"call_id": "call_IDYbPqowlyfI5xuFYBNyLKM9"`

A unique identifier for this particular function call.

Useful if multiple calls happen in parallel and you need to match responses to requests.

3. `"name": "add"`

The function the model has chosen from our toolset. In this case, it correctly picked add.

4. `"type": "function_call"`

Indicates this entry is a function call (not plain text or some other type of output).

5. `"id": "fc_68a959e891ac8194a68ddd1194a9fef90ecfe6806f7bf2f7"`

Another unique identifier, this time for the function call object itself.

6. `"status": "completed"`

Signals that the function call request was fully generated.

## Step 4: Using Function Calling

Earlier, we made a function-calling request to the API even though we hadn't defined any actual functions in our code yet. That worked because the LLM doesn’t directly execute our functions — it just outputs a structured “function call” based on the tool descriptions we give it.

Now it’s time to implement the real functions ourselves. Define the functions that we described earlier in the `tools` dictionary.

> 💡 **Note**: When we define tools for the function calling API, we describe their inputs as JSON objects. That doesn’t mean our Python functions have to accept a single dictionary argument. While you could write add to take a dict and pull values out of it, the more idiomatic approach is to define add just like any normal Python function (e.g. def add(a, b): ...) and then unpack the arguments from the API response before calling the function.

In [ ]:
# TODO: Define the tools we described in `tools`

Now, given our response output and our actual functions, we need to invoke the function with the arguments indicated by our response.

In [ ]:
if len(response.output) == 1 and response.output[0].type == "function_call":
    tool_response = response.output[0]
else:
    raise ValueError(f"Expected exactly one tool response, got {len(response.output)}")

print(tool_response.name)
print(tool_response.arguments)

# TODO: Invoke the function referenced by `function_name` with the arguments referenced by `arguments`
# 💡 Hint: You will need a way to associate the string `"add"` with the function `add`
# 💡 Hint: You will need json.loads for the arguments

result = ???

Now, we have the result from our tool call. When we pass something like "What's 4 times 3?", we're provided with the instructions to invoke `multiply` with the arguments `4` and `3`. We _could_ just output this to our user, but that isn't usually what we want. We're developing chatbots, and we want them to speak like a person! Instead of always responding with something like `12` or `4 * 3 = 12`, we want something like:

> Sure! Four times 3 is 12. Let me know if you need anything else.

To do this, we need to pass the result of our function call _back_ to an LLM as **new context**.

**Note**: You might be thinking, "Isn't this super contrived? LLMs can do basic multiplication." **Yes. It is**. But, let's imagine that LLMs _can't_ do this on their own. If we had more complex calculations, or we were working on this project a couple of years ago, this would be an excellent use case.

In [ ]:
# TODO: Pass the result of our function call back to the LLM
#       along with the user's original request to get a
#       user-friendly response.

## A Complete Example

Finally, let's pull it all together. Define a function, `calculator`, that prompts the user for a natural language operation request, calls the API, and uses the response to invoke the correct function. Finally, pass the result to another LLM call to get a user-friendly response.

In [ ]:
def calculator():
    
    # TODO
    
    pass


calculator()
## Example Usage:
# > Please enter a calculation you'd like to perform (e.g. 'Add five and three'):
# Whats nine divided by 3?
# > Sure! Nine divided by three is 3.0. Let me know if you need anything else.